In [1]:
import pandas as pd
from datetime import datetime
import numpy as np


In [ ]:
#New Script
%run Orders.ipynb
filename = Enn_File_Name
sheetname = 'Data_Source'

In [2]:
#filename = 'Open Orders - 23-Jan'
#sheetname = 'Data_Source'

In [3]:
filedate_str = datetime.strptime(filename[-6:]+'-'+ str(datetime.today().year), '%d-%b-%Y').strftime('%Y%m%d')
print(filedate_str)

20240123


In [4]:
data_gf1 = pd.read_excel(r'E:\_Projects\orders\rcv\\' + filename + '.xlsx', \
                             sheet_name=sheetname, engine='openpyxl', keep_default_na='', \
                            usecols='A:O')

C:\Users\vv185114\AppData\Local\Temp\4\ipykernel_27032\1657527035.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data_gf1 = pd.read_excel(r'E:\_Projects\orders\rcv\\' + filename + '.xlsx', \


In [5]:
tmpl = pd.read_csv('E:\_Projects\orders\map\convert_format_gf1.csv', encoding='utf-8', low_memory=False, keep_default_na='')

In [6]:
cols_dt = ['Create Day', 'Need by date','Actual Promise Date','Original (1st) Promise Date','Update SSD']
for col in data_gf1[cols_dt]:
    data_gf1[col] = pd.to_datetime(data_gf1[col], format='%d-%m-%Y').dt.date

In [7]:
# Fields for Open PO (common for Ennoconn, Jabil)
data_gf1['File'] = 'BUD'
data_gf1['SO'] = data_gf1['S/O number']
data_gf1['Booked Dt'] = data_gf1['Create Day']
data_gf1['PID'] = data_gf1['Material'].str.strip()
data_gf1['Qty'] = data_gf1['Open Qty']
data_gf1['PO'] = data_gf1['PO Number']
# data_gf1['Ship Dt'] = data_gf1['Updated SSD']
data_gf1['PD'] = data_gf1['Actual Promise Date']
data_gf1['Request Dt'] = data_gf1['Need by date']
data_gf1['Tot Qty'] = data_gf1['Order Quantity']
data_gf1['Org PD'] = data_gf1['Original (1st) Promise Date']
data_gf1['Critical'] = ''
data_gf1['Revenue'] = ''
data_gf1['Priority'] = ''
data_gf1['Comments-1'] = ''
data_gf1['Comments-2'] = ''
data_gf1['Customer'] = ''
data_gf1['Prod Cate'] = ''

In [8]:
# Fields for Workbench
data_gf1['Demand'] = 'PO-ENN'
data_gf1['DEMAND VAL'] = data_gf1['PO Number']
conditions = [data_gf1['PID'].str.contains('M')==True, data_gf1['PID'].str.contains('K')==True]
values = ['M','K']
data_gf1['Supply Qty'] = data_gf1['Open Qty']
data_gf1['PID Type'] = np.select(conditions, values, default='')
data_gf1['Supply Detail'] = data_gf1['PO Number']
data_gf1['Whse'] = 'GF1'
data_gf1['Ship Date'] = data_gf1['Update SSD']
data_gf1['PD_PO'] = data_gf1['Actual Promise Date']
data_gf1['IO SSD'] = data_gf1['Ship Date']
data_gf1['Plant Ship date'] = data_gf1['Ship Date']
data_gf1['NEED BY DATE'] = data_gf1['Request Dt']
data_gf1['IO BOOKED DATE'] = data_gf1['Booked Dt']
data_gf1['Plant SSD'] = np.where(data_gf1['PD_PO'].notnull(), data_gf1['PD_PO'], data_gf1['Request Dt'])
data_gf1['Plant SSD'] = np.where(data_gf1['Plant SSD'].isnull(), data_gf1['Ship Date'], data_gf1['Plant SSD'])
data_gf1['Ship Dt'] = data_gf1['Plant SSD']
data_gf1['Validation Flag'] = np.where(data_gf1['PD_PO'].notnull(), 'PD OK', 'NPD')
data_gf1['Supply Type'] = np.where(data_gf1['Ship Date'].notnull(), 'PD', 'PO')
data_gf1['Supplier'] = 'ENNOCONN HUNGARY KFT'
data_gf1 = data_gf1[data_gf1['PID']!='']

In [9]:
data_gf1['Plant SSD'] = data_gf1['Plant SSD'].astype('datetime64[ns]')

In [10]:
data_gf1.to_csv(r'E:\_Projects\_outputs\orders\snd\openpo_enn_v2_' + filedate_str + '.csv', index=False)

In [11]:
data_gf1.to_csv('E:\_Projects\_outputs\orders\snd\workbench_gf1_v2.csv', index=False)